This is a basic Unsupervised structure trainnig.
Place can be reformed: loss func,decoding. not implete val and test yet

# IMPORT

In [1]:
!pip install transformers
!pip install datasets
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [2]:
!git clone https://github.com/strivn/idl-project

Cloning into 'idl-project'...
remote: Enumerating objects: 243, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 243 (delta 16), reused 14 (delta 10), pack-reused 215 (from 1)
Receiving objects: 100% (243/243), 4.55 MiB | 32.35 MiB/s, done.
Resolving deltas: 100% (129/129), done.


In [3]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
import os
import sys
import wandb

In [5]:

wandb.login(key="752c6c684acf41e944cecdf53f1800860e24801d")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA A100-SXM4-40GB


In [14]:
sys.path.append("/content/idl-project")

from src.model import load_fo_model, load_ba_model
from src.data import load_flan_dataset
OUTPUT_DIR = "output/"

if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

# Load model and dataset

In [15]:
fomodel, fotokenizer = load_fo_model()
bamodel, batokenizer = load_ba_model()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fomodel.to(device)
bamodel.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 768)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (query_key_value): Linear(in_features=768, out_features=2304, bias=True)
          (dense): Linear(in_features=768, out_features=768, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=768, out_features=3072, bias=True)
          (dense_4h_to_h): Linear(in_features=3072, out_features=768, bias=True)
          (act): GELUActivation()
        )
      )
    )
    (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwi

In [16]:
batch_size = 16
dataset = load_flan_dataset().batch(batch_size=batch_size)

README.md:   0%|          | 0.00/6.83k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/2167 [00:00<?, ?it/s]

Dataset loaded successfully


In [17]:
print(fotokenizer.pad_token)
print(fotokenizer.pad_token_id)
print(fotokenizer.eos_token)
print(batokenizer.pad_token)
print(batokenizer.pad_token_id)
print(batokenizer.eos_token)

None
None
<|endoftext|>
None
None
None


In [18]:
print("Fo tokenizer added tokens:")
print(fotokenizer.added_tokens_decoder)

print("\nBa tokenizer added tokens:")
print(batokenizer.added_tokens_decoder)

Fo tokenizer added tokens:
{0: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True), 1: AddedToken("<|padding|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True), 50254: AddedToken("                        ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False), 50255: AddedToken("                       ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False), 50256: AddedToken("                      ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False), 50257: AddedToken("                     ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False), 50258: AddedToken("                    ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False), 50259: AddedToken("                   ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False), 50260: Adde

In [19]:
fotokenizer.add_special_tokens({'pad_token': "<|padding|>"})
fotokenizer.pad_token_id = fotokenizer.convert_tokens_to_ids("<|padding|>")
fomodel.config.pad_token_id = fotokenizer.pad_token_id

batokenizer.add_special_tokens({'pad_token': "<pad>"})
batokenizer.pad_token_id = batokenizer.convert_tokens_to_ids("<pad>")
bamodel.config.pad_token_id = batokenizer.pad_token_id


print(fotokenizer.pad_token, fotokenizer.pad_token_id)
print(fotokenizer.eos_token, fotokenizer.eos_token_id)

<|padding|> 1
<|endoftext|> 0


In [70]:
# this preprocess combine preprocess_forward and preprocess_backward
#

def preprocess_dual(example, fo_tokenizer, ba_tokenizer):
    #fo_combined is Q+A. used in regular finetune
    fo_combined = f"{example['inputs']}\n{example['targets']}{fo_tokenizer.eos_token}"
    fo_tok = fo_tokenizer(
        fo_combined,
        truncation=True,
        padding="max_length",
        max_length=768,
        return_tensors="pt"
    )
    fo_input_ids = fo_tok["input_ids"].squeeze(0)
    fo_labels = fo_input_ids.clone()
    fo_labels[:-1] = fo_input_ids[1:]
    fo_labels[fo_input_ids == fo_tokenizer.pad_token_id] = -100
    fo_labels[fo_input_ids == fo_tokenizer.eos_token_id] = -100

    #fo_q_tok is Q only. it will be used in reinforcement study

    fo_q_tok = fo_tokenizer(
        example['inputs'],
        truncation=True,
        padding="max_length",
        max_length=768,
        return_tensors="pt"
    )
    question_only_input_ids = fo_q_tok["input_ids"].squeeze(0)

    #ba_combined is A+Q, will used in regular finetune


    ba_combined = f"{example['targets']}\n{example['inputs']}"
    ba_tok = ba_tokenizer(
        ba_combined,
        truncation=True,
        padding="max_length",
        max_length=768,
        return_tensors="pt"
    )
    ba_input_ids = ba_tok["input_ids"].squeeze(0).flip(dims=[0])
    ba_labels = ba_input_ids.clone()
    ba_labels = torch.roll(ba_labels, shifts=1, dims=0)
    ba_labels[0] = -100

    return {
        "fo_input_ids": fo_input_ids,
        "fo_labels": fo_labels,
        "ba_input_ids": ba_input_ids,
        "ba_labels": ba_labels,
        "question_only_input_ids": question_only_input_ids
    }


In [71]:
examples = next(iter(dataset))

for i in range(3):
    ex = {
        "inputs": examples["inputs"][i],
        "targets": examples["targets"][i]
    }

    dual = preprocess_dual(ex, fotokenizer, batokenizer)

    print(f"===== Example {i+1} =====")

    print("[FO] fo_input_ids shape:", dual["fo_input_ids"].shape)
    print("[FO] fo_labels shape:", dual["fo_labels"].shape)

    print("[BA] ba_input_ids shape:", dual["ba_input_ids"].shape)
    print("[BA] ba_labels shape:", dual["ba_labels"].shape)

    print("[Q ] question_only_input_ids shape:", dual["question_only_input_ids"].shape)


    print("-" * 60)

===== Example 1 =====
[FO] fo_input_ids shape: torch.Size([768])
[FO] fo_labels shape: torch.Size([768])
[BA] ba_input_ids shape: torch.Size([768])
[BA] ba_labels shape: torch.Size([768])
[Q ] question_only_input_ids shape: torch.Size([768])
------------------------------------------------------------
===== Example 2 =====
[FO] fo_input_ids shape: torch.Size([768])
[FO] fo_labels shape: torch.Size([768])
[BA] ba_input_ids shape: torch.Size([768])
[BA] ba_labels shape: torch.Size([768])
[Q ] question_only_input_ids shape: torch.Size([768])
------------------------------------------------------------
===== Example 3 =====
[FO] fo_input_ids shape: torch.Size([768])
[FO] fo_labels shape: torch.Size([768])
[BA] ba_input_ids shape: torch.Size([768])
[BA] ba_labels shape: torch.Size([768])
[Q ] question_only_input_ids shape: torch.Size([768])
------------------------------------------------------------


In [77]:
from torch.utils.data import Dataset, DataLoader

class DualDataset(Dataset):
    def __init__(self, hf_dataset, fo_tokenizer, ba_tokenizer, max_samples=None):
        self.fo_tokenizer = fo_tokenizer
        self.ba_tokenizer = ba_tokenizer

        self.data = []
        for i, example in enumerate(hf_dataset):
            if max_samples and i >= max_samples:
                break
            try:
                processed = preprocess_dual(example, fo_tokenizer, ba_tokenizer)
                self.data.append(processed)
            except:
                continue  # skip error

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

    def get_loader(self, batch_size=8, shuffle=True):
        def collate_fn(batch):
            fo_input_ids = torch.stack([item["fo_input_ids"] for item in batch])
            fo_labels = torch.stack([item["fo_labels"] for item in batch])
            ba_input_ids = torch.stack([item["ba_input_ids"] for item in batch])
            ba_labels = torch.stack([item["ba_labels"] for item in batch])
            question_only_input_ids = torch.stack([
            item["question_only_input_ids"].view(-1)[:768]
            for item in batch
        ])


            return {
                "fo_input_ids": fo_input_ids,
                "fo_labels": fo_labels,
                "ba_input_ids": ba_input_ids,
                "ba_labels": ba_labels,
                "question_only_input_ids": question_only_input_ids
            }
        return DataLoader(
        self,
        batch_size=batch_size,
        shuffle=shuffle,
        collate_fn=collate_fn
    )

In [78]:
dual_dataset = DualDataset(dataset, fotokenizer, batokenizer, max_samples=1000)
loader = dual_dataset.get_loader(batch_size=8)


batch = next(iter(loader))
print("Fo shape:", batch["fo_input_ids"].shape)
print("Ba shape:", batch["ba_input_ids"].shape)
print("Q shape:", batch["question_only_input_ids"].shape)

Fo shape: torch.Size([8, 768])
Ba shape: torch.Size([8, 768])
Q shape: torch.Size([8, 768])


# supervised finetune Train

In [58]:
import torch
import torch.nn as nn
from torch.optim import AdamW
# this code training is actually finetune ba and fo model indenpendly.
# may be can used as a comparasion to the real Unsupervised training
wandb.init(project="dual-training", name="fo-ba-loop-run")
def train_dual(fomodel, fotokenizer, bamodel, batokenizer, dataloader, device, epochs=3):
    fomodel.train()
    bamodel.train()

    fo_optimizer = AdamW(fomodel.parameters(), lr=5e-5)
    ba_optimizer = AdamW(bamodel.parameters(), lr=5e-5)

    loss_fn = nn.CrossEntropyLoss(ignore_index=-100)

    global_step = 0

    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}")
        for batch in dataloader:
            fo_input_ids = batch['fo_input_ids'].to(device)
            fo_labels = batch['fo_labels'].to(device)
            ba_input_ids = batch['ba_input_ids'].to(device)
            ba_labels = batch['ba_labels'].to(device)

            fo_outputs = fomodel(fo_input_ids, labels=fo_labels)
            ba_outputs = bamodel(ba_input_ids, labels=ba_labels)

            fo_loss = fo_outputs.loss
            ba_loss = ba_outputs.loss
            total_loss = fo_loss + ba_loss

            fo_optimizer.zero_grad()
            ba_optimizer.zero_grad()
            total_loss.backward()
            fo_optimizer.step()
            ba_optimizer.step()

            print(f"Loss: {total_loss.item():.4f}")

            wandb.log({
                "global_step": global_step,
                "fo_loss": fo_loss.item(),
                "ba_loss": ba_loss.item(),
                "total_loss": total_loss.item()
            })
            global_step += 1

        torch.save(fomodel.state_dict(), f"{OUTPUT_DIR}/fomodel_epoch{epoch}.pt")
        torch.save(bamodel.state_dict(), f"{OUTPUT_DIR}/bamodel_epoch{epoch}.pt")
        print(f"✅ Saved models for epoch {epoch}")



In [59]:
train_dual(fomodel, fotokenizer, bamodel, batokenizer, loader, device, epochs=3)


Epoch 1
Loss: 23.4519
Loss: 12.8834


KeyboardInterrupt: 

# Generate

In [79]:
def generate_forward(fomodel, tokenizer, input_ids, max_new_tokens=64):
    fomodel.eval()

    if input_ids.dim() == 1:
        input_ids = input_ids.unsqueeze(0)

    input_ids = input_ids.to(fomodel.device)

    pad_token_id = tokenizer.pad_token_id or tokenizer.eos_token_id

    attention_mask = (input_ids != pad_token_id).long().to(fomodel.device)

    with torch.no_grad():
        outputs = fomodel.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            pad_token_id=pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    return outputs

In [82]:

sys.path.append("/content/idl-project")
from src.utils import calculate_score

import torch
import torch.nn as nn
from torch.optim import AdamW

def train_unsupervised(fomodel, fotokenizer, bamodel, batokenizer, dataset, device, epochs=1):

    fomodel.train()
    bamodel.eval()

    optimizer = AdamW(fomodel.parameters(), lr=5e-5)
    global_step = 0

    for epoch in range(epochs):
        print(f"\n[Epoch {epoch+1}]")
        for i, sample in enumerate(dataset):
            question_input_ids = sample['question_only_input_ids'].to(device)
            if question_input_ids.dim() == 1:
                question_input_ids = question_input_ids.unsqueeze(0)

            generated_ids = generate_forward(fomodel, fotokenizer, question_input_ids, max_new_tokens=64)

            query_text = fotokenizer.decode(question_input_ids[0], skip_special_tokens=True)
            answer_text = fotokenizer.decode(generated_ids[0], skip_special_tokens=True)

            try:
                score_result = calculate_score(
                    query=query_text,
                    answer=answer_text,
                    model=bamodel,
                    tokenizer=batokenizer,
                    backward=True,
                    query_direction="reverse"
                )
                loss_value = -score_result['normalized_log_prob']
                loss_tensor = torch.tensor(loss_value, requires_grad=True, device=device)

                optimizer.zero_grad()
                loss_tensor.backward()
                optimizer.step()

                print(f"[Step {global_step}] Loss: {loss_value:.4f} ")
            except Exception as e:
                print(f"[WARN] Skipped sample {i} due to error: {e}")
                continue

            global_step += 1

In [84]:
train_unsupervised(fomodel, fotokenizer, bamodel, batokenizer, loader, device, epochs=2)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



[Epoch 1]


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


[Step 0] Loss: 2.0155 


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

[Step 246] Loss: 2.4986 


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


[Step 247] Loss: 2.6153 


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


[Step 248] Loss: 2.2175 
[Step 249] Loss: 3.3053 
